### MySql Database file
- It covers all database functions


In [1]:
import pandas as pd
import mysql.connector 
from mysql.connector import Error
from sqlalchemy import create_engine

In [2]:
## Create connection
def get_connection():
    return mysql.connector.connect(host='127.0.0.1',
                                   user='root',
                                   password='',
                                   charset="utf8mb4")

In [7]:
## create a Assesment database
def create_db():
    try:
        connection = get_connection()
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("CREATE DATABASE IF NOT EXISTS assesment;")
            print("Database is created...")
            connection.commit()
    except Exception as e:
        print("Error while connecting to MySQL", e)



In [12]:
## create tables 
def create_table():
    try:
        connection = get_connection()
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("USE assesment;")
            #test table with ppr dataset
            # columns: 'Date_of_Sale', 'Address', 'County', 'Eircode', 'Price','Not_Full_Market_Price', 'VAT_Exclusive', 'Description_of_Property','Property_Size_Description'
            cursor.execute(
                """ CREATE TABLE IF NOT EXISTS
                    ppr_test(Date_of_Sale VARCHAR(10),Address VARCHAR(255),County VARCHAR(255),Eircode VARCHAR(10) ,Price VARCHAR(50),Not_Full_Market_Price VARCHAR(5),
                    VAT_Exclusive VARCHAR(10),
                    Description_of_Property VARCHAR(255),
                    Property_Size_Description VARCHAR(255));  """) 
            # comment table
            connection.commit()
            print("Table is created...")
    except Exception as e:
        print("Error while connecting to MySQL", e)

In [4]:
## execute any queries on database
def execute_query(query):
    try:
        connection = get_connection()
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("USE assesment;")
            cursor.execute(query)
            connection.commit()
            print("Query finished")
    except Exception as e:
            print("Query is failed:" + query)

In [5]:
## get query result as a dataframe
def get_data(query):
    try:
        connection = get_connection()
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("USE assesment;")
            cursor.execute(query)
            cols = list(map(lambda x: x[0], cursor.description))
            #result_list=cursor.fetchall()
    except Exception as e:
            print("Query is failed")
    df=pd.DataFrame(cursor.fetchall(),columns=cols)
    return df

In [6]:
## insert data to table from dataframe
def insertData(df,tablename):
    try:
        conn = create_engine("mysql://root@localhost/assesment")
        df.to_sql(tablename,con=conn,if_exists='append',index=False)
        print("Inserting complete...")
    except Exception as e:
        print("Insertin failed :",e)
    

In [8]:
create_db()

Database is created...


In [13]:
create_table()

Table is created...


In [15]:
## test
## read csv file and insert the test dable
ppr="PPR_All.csv"
ppr_df=pd.read_csv(ppr, low_memory=False)
#rename the columns
cols=['Date_of_Sale', 'Address', 'County', 'Eircode', 'Price',
       'Not_Full_Market_Price', 'VAT_Exclusive', 'Description_of_Property',
       'Property_Size_Description']
ppr_df.columns=cols
ppr_df.head()

,Date_of_Sale,Address,County,Eircode,Price,Not_Full_Market_Price,VAT_Exclusive,Description_of_Property,Property_Size_Description
0,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",Dublin,NaN,"343,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
1,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",Laois,NaN,"185,000.00",No,Yes,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin,NaN,"438,500.00",No,No,Second-Hand Dwelling house /Apartment,NaN
3,04/01/2010,"1 The Haven, Mornington",Meath,NaN,"400,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN
4,04/01/2010,"11 Melville Heights, Kilkenny",Kilkenny,NaN,"160,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN


In [16]:
insertData(ppr_df,"ppr_test")

Inserting complete...


In [17]:
# query the table
query="""select * from ppr_test; """
result=get_data(query)
result.head()

,Date_of_Sale,Address,County,Eircode,Price,Not_Full_Market_Price,VAT_Exclusive,Description_of_Property,Property_Size_Description
0,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",Dublin,None,"343,000.00",No,No,Second-Hand Dwelling house /Apartment,None
1,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",Laois,None,"185,000.00",No,Yes,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...
2,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin,None,"438,500.00",No,No,Second-Hand Dwelling house /Apartment,None
3,04/01/2010,"1 The Haven, Mornington",Meath,None,"400,000.00",No,No,Second-Hand Dwelling house /Apartment,None
4,04/01/2010,"11 Melville Heights, Kilkenny",Kilkenny,None,"160,000.00",No,No,Second-Hand Dwelling house /Apartment,None
